In [2]:
import pandas as pd
import pandasql as psql

# Load your data into pandas DataFrames
borough_dim = pd.read_csv('borough_dim.csv')
casualty_dim = pd.read_csv('casualty_dim.csv')
fact_table = pd.read_csv('fact_table.csv')
time_dim = pd.read_csv('time_dim.csv')
weather_dim = pd.read_csv('weather_dim.csv')
light_dim = pd.read_csv('light_dim.csv')

In [20]:
fact_table.shape

(302756, 7)

In [24]:
fact_table.drop_duplicates(subset=['collision_index'])

,collision_index,severity_id,light_conditions_id,weather_conditions_id,Borough_id,casualty_id,time_id
0,2018010080971,3,4,1,E09000007,2018010080971_1,01/01/2018 01:30
2,2018010080973,3,4,1,E09000025,2018010080973_1,01/01/2018 00:50
3,2018010080974,3,4,1,E09000019,2018010080974_1,01/01/2018 00:45
4,2018010080981,2,4,2,E09000025,2018010080981_1,01/01/2018 03:00
5,2018010080982,2,4,1,E09000011,2018010080982_1,01/01/2018 02:20
...,...,...,...,...,...,...,...
302747,1170092586,3,1,1,E09000010,1170092586_1,15/12/2017 15:45
302748,1170092692,3,1,1,E09000010,1170092692_2,28/10/2017 10:20
302752,1170094487,3,2,9,E09000010,1170094487_1,11/11/2017 17:30
302753,1170097147,3,1,1,E09000010,1170097147_1,22/09/2017 07:40


## Question 1: Number of Accidents across Different Boroughs (2013-2023)


In [22]:
result.number_of_accidents.sum()

256257

In [21]:
query = """
SELECT b.borough_name, COUNT(DISTINCT f.collision_index) AS number_of_accidents
FROM fact_table f
JOIN time_dim t ON f.time_id = t.time_id
JOIN borough_dim b ON f.borough_id = b.borough_id
WHERE t.collision_year BETWEEN 2013 AND 2023
GROUP BY b.borough_name
ORDER BY number_of_accidents DESC;
"""
result = psql.sqldf(query, locals())
result

,borough_name,number_of_accidents
0,Westminster,14999
1,Lambeth,12684
2,Tower Hamlets,11062
3,Southwark,10561
4,Wandsworth,10214
5,Ealing,10053
6,Barnet,9983
7,Croydon,9713
8,Brent,9396
9,Enfield,9112


## Question 2: Age Band with the Highest Number of Accidents (Last 10 Years)

In [11]:
query = """
SELECT c.age_band_of_casualty, COUNT(*) AS number_of_accidents
FROM fact_table f
JOIN casualty_dim c ON f.casualty_id = c.casualty_id
JOIN time_dim t ON f.time_id = t.time_id
WHERE t.collision_year >= (SELECT MAX(collision_year) - 10 FROM time_dim)
GROUP BY c.age_band_of_casualty
ORDER BY number_of_accidents DESC
LIMIT 1;
"""
result2 = psql.sqldf(query, locals())
result2

,age_band_of_casualty,number_of_accidents
0,26-35,80761


## Question 3: Influence of Weather & Light Conditions on Accidents (2013-2023)


In [27]:
query = """
SELECT w.weather_conditions, l.light_conditions, COUNT(*) AS number_of_accidents
FROM fact_table f
JOIN weather_dim w ON f.weather_conditions_id = w.weather_conditions_id
JOIN light_dim l ON f.light_conditions_id = l.light_conditions_id
JOIN time_dim t ON f.time_id = t.time_id
WHERE t.collision_year BETWEEN 2013 AND 2023
GROUP BY w.weather_conditions, l.light_conditions
ORDER BY number_of_accidents DESC;
"""
result3 = psql.sqldf(query, locals())
result3

,weather_conditions,light_conditions,number_of_accidents
0,Fine no high winds,Daylight,174314
1,Fine no high winds,Darkness - lights lit,31423
2,Raining no high winds,Daylight,15716
3,Unknown,Daylight,7349
4,Raining no high winds,Darkness - lights lit,7020
5,Other,Daylight,6146
6,Fine no high winds,Darkness - lighting unknown,2255
7,Other,Darkness - lights lit,2086
8,Fine + high winds,Daylight,1438
9,Unknown,Darkness - lights lit,1389


## Question 4: Accident Frequency by Hours of Day & Days of the Week (2013-2023)

In [16]:
query = """
SELECT t.hour, t.day_of_week, COUNT(*) AS number_of_accidents
FROM fact_table f
JOIN time_dim t ON f.time_id = t.time_id
WHERE t.collision_year BETWEEN 2013 AND 2023
GROUP BY t.hour, t.day_of_week
ORDER BY number_of_accidents DESC;
"""
result4 = psql.sqldf(query, locals())
result4

,time,day_of_week,number_of_accidents
0,18:00,Thursday,406
1,18:00,Friday,400
2,17:30,Friday,385
3,18:30,Thursday,385
4,18:00,Wednesday,376
...,...,...,...
13657,9::57,Wednesday,1
13658,9::58,Friday,1
13659,9::58,Wednesday,1
13660,9::59,Monday,1


## Question 5: Trend of the Top 5 Areas with Higher Number of Casualties (2013-2023)

In [29]:
query_5 = """
SELECT collision_year, borough_name, COUNT(*) AS number_of_casualties
FROM (
    SELECT DISTINCT f.collision_index, t.collision_year, b.borough_name
    FROM fact_table f
    JOIN borough_dim b ON f.borough_id = b.borough_id
    JOIN time_dim t ON f.time_id = t.time_id
    WHERE t.collision_year BETWEEN 2013 AND 2023
) sub
GROUP BY collision_year, borough_name
ORDER BY collision_year, number_of_casualties DESC;
"""
result_5 = psql.sqldf(query_5, locals())

# Since pandasql doesn't support window functions like RANK() directly, we need to filter top 5 for each year in Python:
top_5_per_year = result_5.groupby('collision_year').apply(lambda x: x.nlargest(5, 'number_of_casualties')).reset_index(drop=True)
print(top_5_per_year)

    collision_year   borough_name  number_of_casualties
0             2013    Westminster                  1514
1             2013        Lambeth                  1169
2             2013         Barnet                   993
3             2013         Ealing                   946
4             2013  Tower Hamlets                   932
5             2014    Westminster                  1597
6             2014        Lambeth                  1263
7             2014         Ealing                  1046
8             2014  Tower Hamlets                  1044
9             2014      Southwark                  1013
10            2015    Westminster                  1578
11            2015        Lambeth                  1226
12            2015  Tower Hamlets                  1066
13            2015         Barnet                  1062
14            2015         Ealing                  1014
15            2016    Westminster                  1545
16            2016        Lambeth               

## Question 6: Top 5 Most Accident-Affected Areas Each Year (Jul’13-Jun’23)

In [30]:
query_6 = """
SELECT collision_year, borough_name, COUNT(*) AS number_of_accidents
FROM (
    SELECT DISTINCT f.collision_index, t.collision_year, b.borough_name
    FROM fact_table f
    JOIN borough_dim b ON f.borough_id = b.borough_id
    JOIN time_dim t ON f.time_id = t.time_id
    WHERE t.collision_year BETWEEN 2013 AND 2023
) sub
GROUP BY collision_year, borough_name
ORDER BY collision_year, number_of_accidents DESC;
"""
result_6 = psql.sqldf(query_6, locals())

# Filtering for top 5 areas per year in Python:
top_5_areas_per_year = result_6.groupby('collision_year').apply(lambda x: x.nlargest(5, 'number_of_accidents')).reset_index(drop=True)
print(top_5_areas_per_year)

    collision_year   borough_name  number_of_accidents
0             2013    Westminster                 1514
1             2013        Lambeth                 1169
2             2013         Barnet                  993
3             2013         Ealing                  946
4             2013  Tower Hamlets                  932
5             2014    Westminster                 1597
6             2014        Lambeth                 1263
7             2014         Ealing                 1046
8             2014  Tower Hamlets                 1044
9             2014      Southwark                 1013
10            2015    Westminster                 1578
11            2015        Lambeth                 1226
12            2015  Tower Hamlets                 1066
13            2015         Barnet                 1062
14            2015         Ealing                 1014
15            2016    Westminster                 1545
16            2016        Lambeth                 1268
17        

In [9]:
casualty_dim.head()

,casualty_id,age_band_of_casualty,casualty_class,sex_of_casualty,casualty_severity
0,2018010080971_1,46-55,Passenger,Female,Slight
1,2018010080971_2,46-55,Driver/Rider,Male,Slight
2,2018010080973_1,26-35,Pedestrian,Male,Slight
3,2018010080974_1,36-45,Driver/Rider,Male,Slight
4,2018010080981_1,26-35,Driver/Rider,Male,Serious


In [4]:
borough_dim.head()

,borough_id,borough_name
0,E09000001,City of London
1,E09000002,Barking and Dagenham
2,E09000003,Barnet
3,E09000004,Bexley
4,E09000006,Bromley


In [5]:
fact_table.head()

,collision_index,severity_id,light_conditions_id,weather_conditions_id,Borough_id,casualty_id,time_id
0,2018010080971,3,4,1,E09000007,2018010080971_1,01/01/2018 01:30
1,2018010080971,3,4,1,E09000007,2018010080971_2,01/01/2018 01:30
2,2018010080973,3,4,1,E09000025,2018010080973_1,01/01/2018 00:50
3,2018010080974,3,4,1,E09000019,2018010080974_1,01/01/2018 00:45
4,2018010080981,2,4,2,E09000025,2018010080981_1,01/01/2018 03:00


In [7]:
time_dim.head()

,time_id,date,time,collision_year,day_of_week
0,01/01/2018 01:30,01/01/2018,01:30,2018,Monday
1,01/01/2018 00:50,01/01/2018,00:50,2018,Monday
2,01/01/2018 00:45,01/01/2018,00:45,2018,Monday
3,01/01/2018 03:00,01/01/2018,03:00,2018,Monday
4,01/01/2018 02:20,01/01/2018,02:20,2018,Monday


In [12]:
weather_dim.head()

,weather_conditions_id,weather_conditions
0,1,Fine no high winds
1,2,Raining no high winds
2,3,Snowing no high winds
3,4,Fine + high winds
4,5,Raining + high winds


In [14]:
light_dim.head()

,light_conditions_id,light_conditions
0,1,Daylight
1,4,Darkness - lights lit
2,5,Darkness - lights unlit
3,6,Darkness - no lighting
4,7,Darkness - lighting unknown
